In [1]:
import os

from getConfig import getConfig
from MImapping import MImapping

In [2]:
def getFile(file):
    return pd.read_csv(config["dataLoc"]+file, parse_dates=[0], index_col=0)

In [3]:
# This will remove any previously generated columns, leaving only the raw data: date (index) and value
def removeCols(df, col):
    df = pd.DataFrame(df.iloc[:,0])
    df.columns = [col]
    df[col] = round(df[col],2)
    return df

In [4]:
def createMmetrics(df, col):
    s = round(df[col].pct_change(periods=1),3)
    for x in range(4):
        feature = col + "_" + "M%chg_L" + str(x)
        df[feature] = s.shift(x)
    feature = col + "_" + "A%chg" + str(x)
    df[feature] = round(df[col].pct_change(periods=12),3)
    return df

In [5]:
def createQmetrics(df, col):
    s = round(df[col].pct_change(periods=1),3)
    for x in range(3):
        feature = col + "_" + "Q%chg_L" + str(x)
        df[feature] = s.shift(x)
    feature = col + "_" + "A%chg" + str(x)
    df[feature] = round(df[col].pct_change(periods=4),3)
    return df

In [6]:
config  = getConfig()
# This is the mapping of Macro-economic Indicator filename to column and frequency
mapping = MImapping()

In [7]:
for file in os.listdir(config["dataLoc"]):
    if file != "testData.csv":
        df = getFile(file)
        col = mapping[file]["col"]    # This is the name of the column for this MI
        df = removeCols(df, col)
        if mapping[file]["freq"] == "Q":
            df = createQmetrics(df, col)
        else:
            df = createMmetrics(df, col)
        df.to_csv(config["dataLoc"]+file, index=True)